In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math

In [ ]:
@pyimport numpy as np

In [ ]:
include("/home/chuyu/Notebooks/project_place_cell/functions/func_data.jl")

In [ ]:
data_info =   ["20240128_174842", 8, "20240128_182823", 8, "jen"]

data_info =   ["20240127_120909", 1, "20240127_132316", 1, "drew"]

In [ ]:
experiment_filename_1 = data_info[1]
server_1 = data_info[2]

experiment_filename_2 = data_info[3]
server_2 = data_info[4]

experimenter = data_info[end]

In [ ]:
analyzer = "chuyu"

In [ ]:
ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")

In [ ]:
get_synchrony_offset_sweep(experiment_filename_1,server_1; experimenter=experimenter)

In [ ]:
get_synchrony_offset_sweep(experiment_filename_2,server_2; experimenter=experimenter)

In [ ]:
ds_save_analyzer_new = Dataset("$(experiment_filename_1)_$(experiment_filename_2)", analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
mkpath(data_path(ds_save_analyzer_new))

# join neural data

In [ ]:
NMF_filename = joinpath(data_path(ds_save_1), "NMF.h5")
NMF_file = h5open(NMF_filename, "r")

In [ ]:
NMF_filename = joinpath(data_path(ds_save_1), "NMF.h5")
NMF_file = h5open(NMF_filename, "r")
z_all = HDF5.readmmap(NMF_file["z_all"])
centroid_x_all = HDF5.readmmap(NMF_file["centroid_x_all"])
centroid_y_all = HDF5.readmmap(NMF_file["centroid_y_all"])
A_all_1 = HDF5.readmmap(NMF_file["A_all"]);
stack_mean = HDF5.readmmap(NMF_file["stack_mean"]);
close(NMF_file)

NMF_filename = joinpath(data_path(ds_save_2), "NMF.h5")
NMF_file = h5open(NMF_filename, "r")
A_all_2 = HDF5.readmmap(NMF_file["A_all"]);
close(NMF_file)

In [ ]:
A_all = vcat(A_all_1, A_all_2);

In [ ]:
n_frame_1 = size(A_all_1, 1)
n_frame_2 = size(A_all_2, 1)

In [ ]:
joinpath(data_path(ds_save_analyzer_new), "NMF.h5")

In [ ]:
h5open(joinpath(data_path(ds_save_analyzer_new), "NMF.h5"), "w") do file
    file["A_all"] = A_all
    file["centroid_x_all"] = centroid_x_all
    file["centroid_y_all"] = centroid_y_all
    file["z_all"] = z_all
    file["stack_mean"] = stack_mean
end;

# join behavior

In [ ]:
C1, heading1, img_bg1, y_fish1, x_fish1 = h5open(ds_save_1, "behavior.h5"; raw = true) do file
    read(file, "C"),
    read(file, "heading"), 
    read(file, "img_bg"),
    read(file, "fish_anchor_y"), 
    read(file, "fish_anchor_x")
end;


C2, heading2, img_bg2, y_fish2, x_fish2 = h5open(ds_save_2, "behavior.h5"; raw = true) do file
    read(file, "C"),
    read(file, "heading"), 
    read(file, "img_bg"),
    read(file, "fish_anchor_y"), 
    read(file, "fish_anchor_x")
end;

In [ ]:
imshow(img_bg1[:,:,end]', alpha=0.5)
imshow(img_bg2[:,:,end]')

In [ ]:
heading = vcat(heading1[1:125*n_frame_1], heading2[1:125*n_frame_2]);


C = vcat(C1[1:125*n_frame_1], C2[1:125*n_frame_2]);
x_fish = vcat(x_fish1[1:125*n_frame_1], x_fish2[1:125*n_frame_2]);
y_fish = vcat(y_fish1[1:125*n_frame_1], y_fish2[1:125*n_frame_2]);

img_bg = np.concatenate([img_bg1, img_bg2], axis=2);

In [ ]:
mkpath(data_raw_path(ds_save_analyzer_new))

In [ ]:
h5open(joinpath(data_raw_path(ds_save_analyzer_new), "behavior.h5"), "w") do file
    file["heading"] = heading
    file["C"] = C
    file["fish_anchor_x"] = x_fish
    file["fish_anchor_y"] = y_fish
    file["img_bg"] = img_bg
end;

# copy data

In [ ]:

experiment_filename = "20240128_203533"
server = 8
experimenter = "jen"
ds_save = Dataset(experiment_filename, experimenter, gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")
ds_save_cy = Dataset(experiment_filename, "chuyu", gethostname() == "roli-$(server)" ? "/data" : "/nfs/data$(server)")

mkpath(data_path(ds_save_cy))
mkpath(data_raw_path(ds_save_cy))

In [ ]:
which_file = "NMF.h5"
path_origin = joinpath(data_path(ds_save), which_file)
path_target = joinpath(data_path(ds_save_cy), which_file) 
run(`cp -RT $path_origin $path_target`)

In [ ]:
which_file = "registration_experiment.h5"
path_origin = joinpath(data_path(ds_save), which_file)
path_target = joinpath(data_path(ds_save_cy), which_file) 
run(`cp -RT $path_origin $path_target`)

In [ ]:
which_file = "behavior.h5"
path_origin = joinpath(data_raw_path(ds_save), which_file)
path_target = joinpath(data_raw_path(ds_save_cy), which_file) 
run(`cp -RT $path_origin $path_target`)